In [1]:
collection_data = "Spark The Definitive Guid : Big Data Processing Mode Simple"

In [3]:
supplementData = {
    "Spark":1000, 'Definitive':200,'Big':300,'Simple':100
}

In [4]:
# broadcast : 클러스터의 모든 노드에 효율적으로 공유

In [5]:
suppBroadcast =  spark.sparkContext.broadcast(supplementData)

In [6]:
suppBroadcast.value

{'Spark': 1000, 'Definitive': 200, 'Big': 300, 'Simple': 100}

In [19]:
words = spark.sparkContext.parallelize(collection_data.split(),2)  # RDD paprtitioin-->2
words.collect()

['Spark',
 'The',
 'Definitive',
 'Guid',
 ':',
 'Big',
 'Data',
 'Processing',
 'Mode',
 'Simple']

In [18]:
suppBroadcast.value.get('Definitive',0)

200

In [22]:
words.map(lambda word : (word,suppBroadcast.value.get(word,0))).sortBy(lambda x:x[1], ascending=False).collect()

[('Spark', 1000),
 ('Big', 300),
 ('Definitive', 200),
 ('Simple', 100),
 ('The', 0),
 ('Guid', 0),
 (':', 0),
 ('Data', 0),
 ('Processing', 0),
 ('Mode', 0)]

In [23]:
pwd

'/root'

In [26]:
flight = spark.read.parquet("summary.parquet")
flight.show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
+-----------------+-------------------+-----+
only showing top 2 rows



In [27]:
# 항공편추적하기

In [28]:
# 우리나라 항공편 찾기

In [31]:
flight.select('DEST_COUNTRY_NAME').distinct().show()

+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|            Anguilla|
|              Russia|
|            Paraguay|
|             Senegal|
|              Sweden|
|            Kiribati|
|              Guyana|
|         Philippines|
|            Malaysia|
|           Singapore|
|                Fiji|
|              Turkey|
|             Germany|
|         Afghanistan|
|              Jordan|
|               Palau|
|Turks and Caicos ...|
|              France|
|              Greece|
|              Taiwan|
+--------------------+
only showing top 20 rows



In [37]:
from pyspark.sql.functions import col,lower
flight.filter( lower(col('ORIGIN_COUNTRY_NAME')).like("%kor%") ).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|        South Korea|  621|
+-----------------+-------------------+-----+



In [38]:
flight.filter( lower(col('DEST_COUNTRY_NAME')).like("%kor%") ).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|      South Korea|      United States|  683|
+-----------------+-------------------+-----+



In [39]:
# Accumulator 분산환경에서 공유되는 변수 - 여러작업에서 공통적으로 사용되는 값을 수집
# 읽기 전용
# 클러스터의 각 노드에 값을 추가 , 로컬에서는 변경할수 없다
# 디버깅용 정보수집이나 로그작성

In [40]:
# accKor = spark.sparkContext
accKor = sc.accumulator(0)

In [46]:
def accKorFunc(frow):
    dest = frow['DEST_COUNTRY_NAME']
    origin = frow['ORIGIN_COUNTRY_NAME']
    if (dest == 'South Korea') or  (origin == 'South Korea'):
        accKor.add(frow['count'])

flight.foreach(lambda frow : accKorFunc(frow))  





In [48]:
accKor.value

1304

In [50]:
# 신규 세션 생성하기
from pyspark.sql import SparkSession

In [51]:
new_spark = SparkSession.builder.appName("new spark session").master("local[*]")

In [52]:
new_spark

In [53]:
# monitering

In [55]:
ls

2015-summary.csv                    retail-data/
IdeaProjects/                       root
README.md                           sample_libsvm_data.txt
activity-data/                      sample_movielens_ratings.txt
anaconda-ks.cfg                     simple-ml/
apache-hive/                        simple-ml-integers/
bike-data/                          simple-ml-scaling/
binary-classification/              spar_module_ml.ipynb
clustering/                         spark/
data.zip                            spark-3.5.1-bin-hadoop3/
deep-learning-images/               spark-3.5.1-bin-hadoop3.tgz
default.csv                         spark-3.5.1-bin-hadoop3.tgz.1
derby.log                           spark-join.ipynb
doc_log.csv                         spark_share_variable.ipynb
encoded_test.csv                    spart
encoded_test2.csv                   spartk_RDD.ipynb
flight-data/                        sqoop/
flight-data-hive/                   sqoop-1.4.7.bin__hadoop-2.6.0.tar.gz
hadoop/   

In [62]:
spark.read.options(header='true',inferSchema='true').csv('online-dataset.csv').repartition(2)\
.selectExpr("instr(Description , 'GLASS') >=1 as is_glass")\
.groupBy('is_glass').count().show()

+--------+------+
|is_glass| count|
+--------+------+
|    NULL|  1454|
|    true| 12861|
|   false|527594|
+--------+------+



In [63]:
# cashing..... 성능 튜닝

In [64]:
# DataFrame no cashing origin data

In [66]:
ls ./spark/bydata/by-day/

2010-12-01.csv  2011-02-21.csv  2011-05-09.csv  2011-07-20.csv  2011-09-30.csv
2010-12-02.csv  2011-02-22.csv  2011-05-10.csv  2011-07-21.csv  2011-10-02.csv
2010-12-03.csv  2011-02-23.csv  2011-05-11.csv  2011-07-22.csv  2011-10-03.csv
2010-12-05.csv  2011-02-24.csv  2011-05-12.csv  2011-07-24.csv  2011-10-04.csv
2010-12-06.csv  2011-02-25.csv  2011-05-13.csv  2011-07-25.csv  2011-10-05.csv
2010-12-07.csv  2011-02-27.csv  2011-05-15.csv  2011-07-26.csv  2011-10-06.csv
2010-12-08.csv  2011-02-28.csv  2011-05-16.csv  2011-07-27.csv  2011-10-07.csv
2010-12-09.csv  2011-03-01.csv  2011-05-17.csv  2011-07-28.csv  2011-10-09.csv
2010-12-10.csv  2011-03-02.csv  2011-05-18.csv  2011-07-29.csv  2011-10-10.csv
2010-12-12.csv  2011-03-03.csv  2011-05-19.csv  2011-07-31.csv  2011-10-11.csv
2010-12-13.csv  2011-03-04.csv  2011-05-20.csv  2011-08-01.csv  2011-10-12.csv
2010-12-14.csv  2011-03-06.csv  2011-05-22.csv  2011-08-02.csv  2011-10-13.csv
2010-12-15.csv  2011-03-07.csv  2011-05-23.csv  2011

In [73]:
origin_file_path = './spark/bydata/by-day/*.csv'
df1 = spark.read.format('csv').options(header='true',inferSchema = 'true').load(origin_file_path)

In [74]:
df1.count()

541909

In [75]:
df1.show(2)

+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|        Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084| RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077|DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
only showing top 2 rows



In [76]:
# Country -- groupby count -->  df2 save
# df3 CustomerID  count
# df4  InvoiceNo  count
def getCountGroupBy(dataframe,colname):
    return dataframe.groupBy(colname).count().collect()

In [78]:
df2 = getCountGroupBy(df1,'Country')

In [79]:
df3 = getCountGroupBy(df1,'CustomerID')

In [80]:
df4 = getCountGroupBy(df1,'InvoiceNo')

In [83]:
df1.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string]

In [87]:
df1.count()

541909

In [88]:
df2 = getCountGroupBy(df1,'Country')

In [89]:
df3 = getCountGroupBy(df1,'CustomerID')

In [90]:
df4 = getCountGroupBy(df1,'InvoiceNo')

### 이벤트

In [1]:
# 이벤트
spark.conf.set('spark.sql.shuffle.partitions',5)

In [2]:
static = spark.read.json('activity-data/')

In [3]:
streaming = spark.readStream.schema(static.schema).option('maxFilePerTrigger',10).json('activity-data/')

In [4]:
static.show(2)

+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
| Arrival_Time|      Creation_Time|  Device|Index| Model|User|   gt|           x|           y|           z|
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
|1424686735090|1424686733090638193|nexus4_1|   18|nexus4|   g|stand| 3.356934E-4|-5.645752E-4|-0.018814087|
|1424686735292|1424688581345918092|nexus4_2|   66|nexus4|   g|stand|-0.005722046| 0.029083252| 0.005569458|
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
only showing top 2 rows



In [5]:
1e+9

1000000000.0

In [6]:
eventTime = streaming.selectExpr("*", "cast(cast(Creation_Time as double)/1000000000 as timestamp) as event_time")

In [7]:
eventTime

DataFrame[Arrival_Time: bigint, Creation_Time: bigint, Device: string, Index: bigint, Model: string, User: string, gt: string, x: double, y: double, z: double, event_time: timestamp]

In [8]:
from pyspark.sql.functions import col,window
eventTime.groupBy(window(col("event_time"),"10 minutes")).count().writeStream.queryName("pyevents_per_window").format("memory")\
.outputMode("complete").start()

24/03/26 13:36:43 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a782a166-92dd-4cbe-97fd-db782522a37d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/26 13:36:43 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [11]:
from pyspark.sql import SparkSession
new_spark_session = SparkSession.builder.appName("new_spark").getOrCreate()

24/03/26 13:38:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [14]:
from time import sleep
for i in range(10):
    result = new_spark_session.sql("select * from pyevents_per_window").show(3)
    print(result)
    sleep(1)

+--------------------+------+
|              window| count|
+--------------------+------+
|{2015-02-23 19:40...| 88681|
|{2015-02-24 20:50...|150773|
|{2015-02-24 22:00...|133323|
+--------------------+------+
only showing top 3 rows

None
+--------------------+------+
|              window| count|
+--------------------+------+
|{2015-02-23 19:40...| 88681|
|{2015-02-24 20:50...|150773|
|{2015-02-24 22:00...|133323|
+--------------------+------+
only showing top 3 rows

None
+--------------------+------+
|              window| count|
+--------------------+------+
|{2015-02-23 19:40...| 88681|
|{2015-02-24 20:50...|150773|
|{2015-02-24 22:00...|133323|
+--------------------+------+
only showing top 3 rows

None
+--------------------+------+
|              window| count|
+--------------------+------+
|{2015-02-23 19:40...| 88681|
|{2015-02-24 20:50...|150773|
|{2015-02-24 22:00...|133323|
+--------------------+------+
only showing top 3 rows

None
+--------------------+------+
|         

In [114]:
eventTime.groupBy(window(col("event_time"),"10 minutes","5 minutes")).count().writeStream.queryName("pyevents_per_window2").format("memory")\
.outputMode("complete").start()

24/03/26 13:23:48 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-86996183-9c6d-4faf-a9cb-6951b0a1b5bd. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/26 13:23:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [124]:
eventTime.withWatermark("event_time","30 minutes")\
.groupBy(window(col("event_time"),"10 minutes","5 minutes")).count().writeStream.queryName("pyevents_per_window3").format("memory")\
.outputMode("complete").start()

ConnectionRefusedError: [Errno 111] Connection refused

In [123]:
eventTime.withWatermark("event_time","5 seconds")\
.dropDuplicates(["User","event_time"])\
.groupBy("User").count().writeStream.queryName("pyevents_per_window4").format("memory")\
.outputMode("complete").start()

ConnectionRefusedError: [Errno 111] Connection refused